In [1]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 500)

In [2]:
df_list = []
filename = 'C:/Users/sebks/PycharmProjects/Real-Time-Load-Management-Soccer/data/19060518.dat'
with open(filename) as file:
    lines = file.readlines()
    print(lines[0].split(':')[0])
    for line in lines:
        team = line.split(':')[1].split(';')
        one_row = ','.join(map(str, team)).split(',')
        one_row = one_row[:-1]
        df_list.append(one_row)

1821287


In [3]:
df = pd.DataFrame(df_list)

cols = ['team', 'target_id', 'number', 'x', 'y', 'speed']

final_cols = []
for i in range(1,int(len(df.columns)/6)+1):
    for attr in cols:
        var = attr + "_" + str(i)
        final_cols.append(var)
        
df.columns = final_cols

team = [col for col in df.columns if 'team' in col]
target_id = [col for col in df.columns if 'target_id' in col]
number = [col for col in df.columns if 'number' in col]
x = [col for col in df.columns if 'x' in col]
y = [col for col in df.columns if 'y' in col]
speed = [col for col in df.columns if 'speed' in col]

print(team)

df3 = pd.lreshape(df, {'team' : team, 'target_id' : target_id, 'jersey_number': number, 'x':x, 'y':y, 'speed':speed})
number_list = list(range(1821287,len(df)+1821287))
df3['id'] = np.tile(number_list, len(df3)//len(number_list))

df4 = df3[(df3['team'] != str(-1)) & (df3['team'] != str(4))]
df4['id'] = df4['id'].astype(int)

['team_1', 'team_2', 'team_3', 'team_4', 'team_5', 'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11', 'team_12', 'team_13', 'team_14', 'team_15', 'team_16', 'team_17', 'team_18', 'team_19', 'team_20', 'team_21', 'team_22', 'team_23', 'team_24', 'team_25', 'team_26', 'team_27', 'team_28', 'team_29']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
final_df = pd.DataFrame()
meta_data = "C:/Users/sebks/PycharmProjects/Real-Time-Load-Management-Soccer/data/19060518_metadata.xml"
with open(meta_data, 'r') as f:
    data = f.read()
    
soup = BeautifulSoup(data, "xml")
period = soup.find_all('period')
for half in range(0,2):
    start = period[half]['iStartFrame']
    end = period[half]['iEndFrame']
    half_df = df4[(df4['id'] >= int(start)) & (df4['id'] <= int(end))]
    half_df['half'] = half
    final_df = pd.concat([final_df, half_df])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [5]:
final_df = final_df.reset_index(drop=True)
start_time = '2019-06-05T' + soup.find_all('match')[0]['dtDate']
final_df.insert(0, 'timestamp', pd.to_datetime(start_time))

In [6]:
pd.to_datetime(final_df.iloc[0, 0]) + pd.DateOffset(seconds=3.3)

Timestamp('2019-06-05 20:45:03.300000')

In [7]:
# get frames per second from metadata
fps = int(soup.find_all('match')[0]['iFrameRateFps'])

# sort df by id
sorted_df = final_df.sort_values(by='id')
sorted_df['order'] = sorted_df.index

# get first row of each id
first_values = sorted_df.groupby('id').first()['order']
first_values = list(first_values.values)[1:]

# set value where the time increases because the frame count increases
sorted_df.insert(len(sorted_df.columns), 'time_diff', 0)
sorted_df.loc[sorted_df['order'].isin(first_values), 'time_diff'] = 1/fps
sorted_df['new_id'] = sorted_df['id'] - 1870230

# multiply the time diff with the number of increased frames
sorted_df['new_time_diff'] = sorted_df['time_diff'] * sorted_df['new_id']
# forward fill the time_diff
sorted_df['new_time_diff'] = sorted_df['new_time_diff'].replace(to_replace=0.0, method='ffill')
# create the new timestamps so that they matches the frame count
sorted_df['timestamp'] = pd.to_datetime(start_time) + pd.to_timedelta(sorted_df['new_time_diff'], unit='S')

In [13]:
export_df = sorted_df[['timestamp', 'team', 'jersey_number', 'x', 'y', 'speed', 'half']]

In [14]:
export_df = export_df[export_df['team'] != str(3)]
export_df = export_df.reset_index(drop=True)

In [15]:
export_df['timestamp'] = export_df['timestamp'].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
export_df['timestamp'] = export_df['timestamp'].astype(str).str[:-3]
export_df['timestamp'] = export_df['timestamp'] + "+0100"

In [16]:
export_df['teamjersey'] = export_df['jersey_number'] + export_df['team']
export_df['teamjersey'] = export_df['teamjersey'].astype(int)
export_df['player_id'] = export_df['teamjersey'].rank(method='dense').astype(int)

In [37]:
export_df[export_df['player_id'] == 28]

,timestamp,team,jersey_number,x,y,speed,half,teamjersey,player_id
11,2019-06-05T20:45:00.000+0100,1,23,43,-1478,1.39,0,231,28
27,2019-06-05T20:45:00.040+0100,1,23,50,-1476,1.49,0,231,28
49,2019-06-05T20:45:00.080+0100,1,23,57,-1474,1.58,0,231,28
73,2019-06-05T20:45:00.120+0100,1,23,64,-1473,1.72,0,231,28
91,2019-06-05T20:45:00.160+0100,1,23,72,-1472,1.80,0,231,28
...,...,...,...,...,...,...,...,...,...
2342567,2019-06-05T22:11:04.720+0100,1,23,29,-3371,0.98,1,231,28
2342597,2019-06-05T22:11:04.760+0100,1,23,30,-3371,0.57,1,231,28
2342620,2019-06-05T22:11:04.800+0100,1,23,31,-3372,0.34,1,231,28
2342642,2019-06-05T22:11:04.840+0100,1,23,27,-3374,0.34,1,231,28


In [38]:
export_df.to_csv('tracking_data_id.csv', index=False)